## Step 1: Verify Environment and GPU Configuration

Validates dual T4 GPU setup, CUDA version compatibility, and VRAM availability to ensure optimal configuration for advanced llama-server deployment scenarios.

In [1]:
import subprocess
import os

print("="*70)
print("🔍 ENVIRONMENT CHECK")
print("="*70)

# GPU check
result = subprocess.run(["nvidia-smi", "--query-gpu=index,name,memory.total,compute_cap", 
                         "--format=csv,noheader"], capture_output=True, text=True)
print("\n📊 GPUs Available:")
for line in result.stdout.strip().split('\n'):
    print(f"   {line}")

# CUDA version
print("\n📊 CUDA Version:")
!nvcc --version | grep release

print("\n✅ Environment ready for llama-server configuration")

🔍 ENVIRONMENT CHECK

📊 GPUs Available:
   0, Tesla T4, 15360 MiB, 7.5
   1, Tesla T4, 15360 MiB, 7.5

📊 CUDA Version:
Cuda compilation tools, release 12.5, V12.5.82

✅ Environment ready for llama-server configuration


## Step 2: Install llamatelemetry and Dependencies

Installs llamatelemetry v0.1.0 with forced cache refresh to ensure latest binaries, plus HuggingFace Hub and SSE client for streaming support.

In [2]:
%%time
# Install llamatelemetry v0.1.0 (force fresh install to ensure correct binaries)
!pip install -q --no-cache-dir --force-reinstall git+https://github.com/llamatelemetry/llamatelemetry.git@v0.1.0
!pip install -q huggingface_hub sseclient-py

import llamatelemetry
print(f"✅ llamatelemetry {llamatelemetry.__version__} installed")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 255.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 278.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 339.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 331.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 328.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 340.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 383.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 268.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/20


🎯 llamatelemetry v0.1.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llamatelemetry v0.1.0!
🌐 Platform: Colab

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v0.1.0 from HuggingFace Hub...
   Repo: waqasm86/llamatelemetry-binaries
   File: v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(


v0.1.0/llamatelemetry-v0.1.0-cuda12-kagg(…):   0%|          | 0.00/1.40G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llamatelemetry/extract_0.1.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llamatelemetry/extract_0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12
  Copied 2 libraries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/lib
✅ Binaries installed successfully!

✅ llamatelemetry 0.1.0 installed
CPU times: user 48 s, sys: 11.4 s, total: 59.4 s
Wall time: 2min 8s


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Graphistry_Personal_Key_ID")
secret_value_1 = user_secrets.get_secret("Graphistry_Personal_Secret_Key")
hf_token = user_secrets.get_secret("HF_TOKEN")

In [5]:
from huggingface_hub import login
import os

# Login to Hugging Face
try:
    login(token=hf_token)
    print("Successfully logged into Hugging Face!")
except Exception as e:
    print(f"Error logging into Hugging Face: {e}")

Successfully logged into Hugging Face!


## Step 3: Explore Server Configuration Options

Comprehensive overview of llama-server configuration flags covering model settings, GPU allocation, performance tuning, and network parameters for production deployments.

In [6]:
from llamatelemetry.server import ServerManager
from llamatelemetry.api.multigpu import MultiGPUConfig, SplitMode

# Display all configuration options
print("="*70)
print("📋 LLAMA-SERVER CONFIGURATION OPTIONS")
print("="*70)

config_options = {
    "Model Settings": {
        "--model, -m": "Path to GGUF model file",
        "--alias, -a": "Model alias for API responses",
        "--ctx-size, -c": "Context size (default: 4096)",
        "--batch-size, -b": "Batch size for prompt processing",
        "--ubatch-size": "Physical batch size (default: 512)",
    },
    "GPU Settings": {
        "--n-gpu-layers, -ngl": "Layers to offload to GPU (99 = all)",
        "--main-gpu, -mg": "Main GPU for computations",
        "--tensor-split, -ts": "VRAM distribution across GPUs",
        "--split-mode, -sm": "Split mode: layer, row, none",
    },
    "Performance": {
        "--flash-attn, -fa": "Enable FlashAttention (faster)",
        "--threads, -t": "CPU threads for generation",
        "--threads-batch, -tb": "CPU threads for batch processing",
        "--cont-batching": "Enable continuous batching",
        "--parallel, -np": "Number of parallel sequences",
    },
    "Server Settings": {
        "--host": "Host address (default: 127.0.0.1)",
        "--port": "Port number (default: 8080)",
        "--timeout": "Server timeout in seconds",
        "--embeddings": "Enable embeddings endpoint",
    },
}

for category, options in config_options.items():
    print(f"\n📌 {category}:")
    for flag, desc in options.items():
        print(f"   {flag:25} {desc}")

📋 LLAMA-SERVER CONFIGURATION OPTIONS

📌 Model Settings:
   --model, -m               Path to GGUF model file
   --alias, -a               Model alias for API responses
   --ctx-size, -c            Context size (default: 4096)
   --batch-size, -b          Batch size for prompt processing
   --ubatch-size             Physical batch size (default: 512)

📌 GPU Settings:
   --n-gpu-layers, -ngl      Layers to offload to GPU (99 = all)
   --main-gpu, -mg           Main GPU for computations
   --tensor-split, -ts       VRAM distribution across GPUs
   --split-mode, -sm         Split mode: layer, row, none

📌 Performance:
   --flash-attn, -fa         Enable FlashAttention (faster)
   --threads, -t             CPU threads for generation
   --threads-batch, -tb      CPU threads for batch processing
   --cont-batching           Enable continuous batching
   --parallel, -np           Number of parallel sequences

📌 Server Settings:
   --host                    Host address (default: 127.0.0.1)
   

## Step 4: Review Kaggle T4 Configuration Presets

Pre-configured settings for single T4 (15GB), dual T4 (30GB), and split-GPU modes with optimized context sizes, batch parameters, and FlashAttention settings.

In [8]:
# Step 4 (fixed for llamatelemetry v0.1.0)

from llamatelemetry.api.multigpu import (
    kaggle_t4_dual_config,
    colab_t4_single_config,
    auto_config,
    detect_gpus,
)

print("="*70)
print("📋 KAGGLE T4 CONFIGURATION PRESETS")
print("="*70)

# Single GPU configuration (use GPU 0 only)
print("\n🔹 Single T4 Configuration (15GB VRAM):")
single_config = colab_t4_single_config()
print(f"   GPU Layers: {single_config.n_gpu_layers}")
print(f"   Context Size: {single_config.ctx_size}")
print(f"   Batch Size: {single_config.batch_size}")
print(f"   Micro-batch Size: {single_config.ubatch_size}")
print(f"   Flash Attention: {single_config.flash_attention}")
print(f"   Split Mode: {single_config.split_mode.value}")
print("   Best for: Models up to ~7B Q4_K_M")

# Dual GPU configuration (split across both GPUs)
print("\n🔹 Dual T4 Configuration (30GB VRAM total):")
dual_config = kaggle_t4_dual_config()
print(f"   GPU Layers: {dual_config.n_gpu_layers}")
print(f"   Context Size: {dual_config.ctx_size}")
print(f"   Batch Size: {dual_config.batch_size}")
print(f"   Micro-batch Size: {dual_config.ubatch_size}")
print(f"   Tensor Split: {dual_config.tensor_split}")
print(f"   Split Mode: {dual_config.split_mode.value}")
print(f"   Flash Attention: {dual_config.flash_attention}")
print("   Best for: Models up to ~13B Q4_K_M")

# Auto-detect configuration (optional)
print("\n🔹 Auto-Detected Configuration:")
auto_cfg = auto_config()
print(f"   GPU Layers: {auto_cfg.n_gpu_layers}")
print(f"   Context Size: {auto_cfg.ctx_size}")
print(f"   Tensor Split: {auto_cfg.tensor_split}")
print(f"   Split Mode: {auto_cfg.split_mode.value}")
print(f"   Flash Attention: {auto_cfg.flash_attention}")

# Split GPU note
print("\n🔹 Split-GPU Configuration (Recommended):")
print("   GPU 0: llama-server (LLM inference)")
print("   GPU 1: RAPIDS/Graphistry (graph processing)")
print("   Best for: Combined LLM + visualization workflows")


📋 KAGGLE T4 CONFIGURATION PRESETS

🔹 Single T4 Configuration (15GB VRAM):
   GPU Layers: -1
   Context Size: 4096
   Batch Size: 1024
   Micro-batch Size: 256
   Flash Attention: True
   Split Mode: none
   Best for: Models up to ~7B Q4_K_M

🔹 Dual T4 Configuration (30GB VRAM total):
   GPU Layers: -1
   Context Size: 8192
   Batch Size: 2048
   Micro-batch Size: 512
   Tensor Split: [0.5, 0.5]
   Split Mode: layer
   Flash Attention: True
   Best for: Models up to ~13B Q4_K_M

🔹 Auto-Detected Configuration:
   GPU Layers: -1
   Context Size: 8192
   Tensor Split: [0.5, 0.5]
   Split Mode: layer
   Flash Attention: True

🔹 Split-GPU Configuration (Recommended):
   GPU 0: llama-server (LLM inference)
   GPU 1: RAPIDS/Graphistry (graph processing)
   Best for: Combined LLM + visualization workflows


## Step 5: Download Test Model

Downloads Gemma 3-1B Instruct Q4_K_M (~750MB) for testing various server configurations with minimal VRAM requirements and fast loading times.

In [9]:
%%time
from huggingface_hub import hf_hub_download
import os

# Download a small model for testing configurations
MODEL_REPO = "unsloth/gemma-3-1b-it-GGUF"
MODEL_FILE = "gemma-3-1b-it-Q4_K_M.gguf"

print(f"📥 Downloading {MODEL_FILE} for configuration testing...")

model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir="/kaggle/working/models"
)

size_gb = os.path.getsize(model_path) / (1024**3)
print(f"\n✅ Model downloaded: {model_path}")
print(f"   Size: {size_gb:.2f} GB")

📥 Downloading gemma-3-1b-it-Q4_K_M.gguf for configuration testing...


gemma-3-1b-it-Q4_K_M.gguf:   0%|          | 0.00/806M [00:00<?, ?B/s]


✅ Model downloaded: /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf
   Size: 0.75 GB
CPU times: user 1.71 s, sys: 3.02 s, total: 4.73 s
Wall time: 2.85 s


## Step 6: Launch Server with Basic Configuration

Starts llama-server with fundamental settings including GPU layer offloading, context size, and batch parameters for baseline performance testing.

In [10]:
from llamatelemetry.server import ServerManager

# Create basic configuration settings (used as parameters to start_server)
print("="*70)
print("🔧 BASIC SERVER CONFIGURATION")
print("="*70)

# Configuration parameters
config = {
    "model_path": model_path,
    "host": "127.0.0.1",
    "port": 8080,
    "gpu_layers": 99,       # Offload all layers to GPU
    "ctx_size": 4096,       # 4K context
    "batch_size": 512,      # Batch size for prompt processing
}

print(f"\n📋 Configuration:")
print(f"   Model: {config['model_path']}")
print(f"   Host: {config['host']}:{config['port']}")
print(f"   GPU Layers: {config['gpu_layers']}")
print(f"   Context: {config['ctx_size']}")

# Start server using ServerManager.start_server() API
server = ServerManager(server_url=f"http://{config['host']}:{config['port']}")
print("\n🚀 Starting server with basic configuration...")

try:
    server.start_server(
        model_path=config['model_path'],
        host=config['host'],
        port=config['port'],
        gpu_layers=config['gpu_layers'],
        ctx_size=config['ctx_size'],
        timeout=60,
        verbose=True
    )
    print("\n✅ Server started successfully!")
except Exception as e:
    print(f"\n❌ Server failed to start: {e}")

🔧 BASIC SERVER CONFIGURATION

📋 Configuration:
   Model: /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf
   Host: 127.0.0.1:8080
   GPU Layers: 99
   Context: 4096

🚀 Starting server with basic configuration...
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: gemma-3-1b-it-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8080
Waiting for server to be ready....... ✓ Ready in 4.0s

✅ Server started successfully!


## Step 6.5: Wait for server readiness + sanity checks

In [13]:
# Step 6.5: Wait for readiness + quick inference sanity test

import time
import os
import requests

print("="*70)
print("⏳ WAIT FOR SERVER READY + SANITY TEST")
print("="*70)

# Confirm model file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found: {model_path}")

base_url = server.server_url if hasattr(server, "server_url") else "http://127.0.0.1:8080"

# Poll health endpoint
ready = False
for i in range(60):
    try:
        r = requests.get(f"{base_url}/health", timeout=2)
        if r.status_code == 200:
            print(f"✅ Server ready ({i+1}s): {r.json()}")
            ready = True
            break
    except Exception:
        pass
    time.sleep(1)

if not ready:
    print("❌ Server not ready after 60s")
    info = server.get_server_info() if hasattr(server, "get_server_info") else {}
    print(f"Server info: {info}")
    if getattr(server, "server_process", None) is not None:
        print(f"Process running: {server.server_process.poll() is None}")
    raise RuntimeError("llama-server did not become ready")

# Quick props check
try:
    props = requests.get(f"{base_url}/props", timeout=5).json()
    print(f"\n📌 Loaded model path: {props.get('model_path', 'N/A')}")
    print(f"📌 Context size: {props.get('default_generation_settings', {}).get('n_ctx', 'N/A')}")
except Exception as e:
    print(f"Props check skipped: {e}")

# Quick inference sanity test
print("\n🧪 Running quick test completion...")
payload = {
    "prompt": "Hello! In one sentence, what is llamatelemetry?",
    "n_predict": 40,
    "temperature": 0.7,
    "top_p": 0.9,
    "top_k": 40,
    "stream": False
}

try:
    resp = requests.post(f"{base_url}/completion", json=payload, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    print("✅ Completion OK")
    print("Response:", data.get("content", "").strip())
except Exception as e:
    print(f"❌ Completion failed: {e}")


⏳ WAIT FOR SERVER READY + SANITY TEST
✅ Server ready (1s): {'status': 'ok'}

📌 Loaded model path: /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf
📌 Context size: 4096

🧪 Running quick test completion...
✅ Completion OK
Response: Llamatelemetry is a process of analyzing and extracting insights from a company's call center data to improve its customer service processes.

Do you want to learn more about this topic?


## Step 7: Monitor Server Health and Status

Checks server health endpoint, model properties, and slot availability to verify successful startup and readiness for inference requests.

In [15]:
# Step 7 (updated): Server Health Monitoring + Live Slot Activity

import time
import requests

print("="*70)
print("🏥 SERVER HEALTH MONITORING")
print("="*70)

base_url = "http://127.0.0.1:8080"

def safe_json(resp):
    try:
        return resp.json()
    except Exception:
        return resp.text

# 1) Health check
try:
    health = requests.get(f"{base_url}/health", timeout=5)
    print(f"\n📊 Health Status ({health.status_code}): {safe_json(health)}")
except Exception as e:
    print(f"❌ Health check failed: {e}")

# 2) Model properties
try:
    props = requests.get(f"{base_url}/props", timeout=5)
    data = safe_json(props)
    print(f"\n📊 Model Properties ({props.status_code}):")
    if isinstance(data, dict):
        print(f"   Model path: {data.get('model_path', 'N/A')}")
        print(f"   Context: {data.get('default_generation_settings', {}).get('n_ctx', 'N/A')}")
        print(f"   Total slots: {data.get('total_slots', 'N/A')}")
    else:
        print(f"   Raw: {data}")
except Exception as e:
    print(f"   Props endpoint not available: {e}")

# 3) Slots (correct fields)
def print_slots(label="Server Slots"):
    try:
        slots = requests.get(f"{base_url}/slots", timeout=5).json()
        print(f"\n📊 {label}:")
        for slot in slots:
            slot_id = slot.get("id", "N/A")
            is_processing = slot.get("is_processing", "N/A")
            n_ctx = slot.get("n_ctx", "N/A")
            n_predict = slot.get("n_predict", "N/A")
            print(f"   Slot {slot_id}: processing={is_processing}, n_ctx={n_ctx}, n_predict={n_predict}")
    except Exception as e:
        print(f"   Slots endpoint not available: {e}")

print_slots("Server Slots (Idle Check)")

# 4) Live slot activity check
print("\n🧪 Live slot activity test (short completion)...")
payload = {
    "prompt": "Say hello in one short sentence.",
    "n_predict": 20,
    "temperature": 0.7,
    "top_p": 0.9,
    "top_k": 40,
    "stream": False
}

try:
    # Fire request and check slots while running
    req = requests.post(f"{base_url}/completion", json=payload, timeout=60)
    req.raise_for_status()
    time.sleep(0.5)
    print_slots("Server Slots (After Test Completion)")
    print("✅ Test completion OK")
except Exception as e:
    print(f"❌ Test completion failed: {e}")


🏥 SERVER HEALTH MONITORING

📊 Health Status (200): {'status': 'ok'}

📊 Model Properties (200):
   Model path: /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf
   Context: 4096
   Total slots: 1

📊 Server Slots (Idle Check):
   Slot 0: processing=False, n_ctx=4096, n_predict=N/A

🧪 Live slot activity test (short completion)...

📊 Server Slots (After Test Completion):
   Slot 0: processing=False, n_ctx=4096, n_predict=N/A
✅ Test completion OK


## Step 8: Stop Server for Reconfiguration

Gracefully stops current server instance and waits for port release to prepare for testing advanced configuration scenarios.

In [16]:
# Stop current server
print("🛑 Stopping current server...")
server.stop_server()

import time
time.sleep(2)  # Wait for port to be released

print("\n✅ Server stopped")

🛑 Stopping current server...

✅ Server stopped


## Step 9: Deploy High-Performance Configuration

Launches server with optimized settings for maximum throughput including larger context, increased batch size, and parallel slot configuration for concurrent requests.

In [17]:
print("="*70)
print("⚡ HIGH-PERFORMANCE CONFIGURATION")
print("="*70)

# High-performance configuration parameters
hp_config = {
    "model_path": model_path,
    "host": "127.0.0.1",
    "port": 8080,
    
    # GPU settings - maximize GPU utilization
    "gpu_layers": 99,
    
    # Context and batching
    "ctx_size": 8192,      # Larger context
    "batch_size": 1024,    # Larger batch for prompt processing
    "ubatch_size": 512,    # Physical batch size
    
    # Parallelism
    "n_parallel": 4,       # 4 parallel request slots
}

print(f"\n📋 High-Performance Settings:")
print(f"   Context Size: {hp_config['ctx_size']} tokens")
print(f"   Batch Size: {hp_config['batch_size']}")
print(f"   Parallel Slots: {hp_config['n_parallel']}")

# Create new server manager
server = ServerManager(server_url=f"http://{hp_config['host']}:{hp_config['port']}")

# Start with high-performance config
print("\n🚀 Starting server with high-performance configuration...")
try:
    server.start_server(
        model_path=hp_config['model_path'],
        host=hp_config['host'],
        port=hp_config['port'],
        gpu_layers=hp_config['gpu_layers'],
        ctx_size=hp_config['ctx_size'],
        batch_size=hp_config['batch_size'],
        ubatch_size=hp_config['ubatch_size'],
        n_parallel=hp_config['n_parallel'],
        timeout=60,
        verbose=True
    )
    print("\n✅ High-performance server started!")
except Exception as e:
    print(f"\n❌ Server failed to start: {e}")

⚡ HIGH-PERFORMANCE CONFIGURATION

📋 High-Performance Settings:
   Context Size: 8192 tokens
   Batch Size: 1024
   Parallel Slots: 4

🚀 Starting server with high-performance configuration...
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: gemma-3-1b-it-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 8192
  Server URL: http://127.0.0.1:8080
Waiting for server to be ready...... ✓ Ready in 3.0s

✅ High-performance server started!


## Step 10: Benchmark Inference Performance

Runs multiple test prompts to measure tokens-per-second generation speed, validating performance improvements from optimized configuration settings.

In [18]:
import time
from llamatelemetry.api.client import LlamaCppClient

print("="*70)
print("📊 INFERENCE PERFORMANCE BENCHMARK")
print("="*70)

client = LlamaCppClient(base_url="http://127.0.0.1:8080")

# Benchmark parameters
prompts = [
    "Explain quantum computing in simple terms.",
    "Write a haiku about machine learning.",
    "What are the benefits of GPU acceleration?",
]

print("\n🏃 Running benchmark with 3 prompts...\n")

total_tokens = 0
total_time = 0

for i, prompt in enumerate(prompts, 1):
    start = time.time()
    
    response = client.chat.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7
    )
    
    elapsed = time.time() - start
    tokens = response.usage.completion_tokens
    
    total_tokens += tokens
    total_time += elapsed
    
    print(f"   Prompt {i}: {tokens} tokens in {elapsed:.2f}s ({tokens/elapsed:.1f} tok/s)")

print(f"\n📊 Benchmark Results:")
print(f"   Total Tokens: {total_tokens}")
print(f"   Total Time: {total_time:.2f}s")
print(f"   Average Speed: {total_tokens/total_time:.1f} tokens/second")

📊 INFERENCE PERFORMANCE BENCHMARK

🏃 Running benchmark with 3 prompts...

   Prompt 1: 100 tokens in 1.58s (63.3 tok/s)
   Prompt 2: 20 tokens in 0.32s (62.9 tok/s)
   Prompt 3: 100 tokens in 1.50s (66.5 tok/s)

📊 Benchmark Results:
   Total Tokens: 220
   Total Time: 3.40s
   Average Speed: 64.7 tokens/second


## Step 11: Monitor GPU Memory Allocation

Tracks VRAM usage across both GPUs using nvidia-smi to understand memory footprint and validate efficient resource utilization.

In [19]:
print("="*70)
print("📊 GPU MEMORY MONITORING")
print("="*70)

# Current memory usage
print("\n🔹 Current GPU Memory Usage:")
!nvidia-smi --query-gpu=index,name,memory.used,memory.total,memory.free --format=csv

# Memory usage over time (single snapshot)
import subprocess
result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,memory.used,utilization.gpu", "--format=csv,noheader"],
    capture_output=True, text=True
)

print("\n🔹 GPU Utilization:")
for line in result.stdout.strip().split('\n'):
    parts = line.split(', ')
    if len(parts) >= 3:
        print(f"   GPU {parts[0]}: {parts[1]} used, {parts[2]} utilization")

📊 GPU MEMORY MONITORING

🔹 Current GPU Memory Usage:
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
index, name, memory.used [MiB], memory.total [MiB], memory.free [MiB]
0, Tesla T4, 509 MiB, 15360 MiB, 14404 MiB
1, Tesla T4, 1229 MiB, 15360 MiB, 13684 MiB

🔹 GPU Utilization:
   GPU 0: 509 MiB used, 0 % utilization
   GPU 1: 1229 MiB used, 0 % utilization


## Step 12: Command-Line Reference Guide

Provides comprehensive CLI examples for running llama-server directly with various configurations including single GPU, dual GPU, high-performance, and embeddings mode.

In [20]:
print("="*70)
print("📋 COMMAND-LINE REFERENCE")
print("="*70)

cli_examples = f"""
🔹 Basic Start:
   llama-server -m {model_path} --host 0.0.0.0 --port 8080

🔹 Single GPU (GPU 0, 15GB):
   llama-server -m {model_path} \\
       --host 0.0.0.0 --port 8080 \\
       --n-gpu-layers 99 --main-gpu 0 \\
       --ctx-size 4096 --flash-attn

🔹 Dual GPU (30GB total):
   llama-server -m {model_path} \\
       --host 0.0.0.0 --port 8080 \\
       --n-gpu-layers 99 \\
       --tensor-split 0.5,0.5 \\
       --split-mode layer \\
       --ctx-size 8192 --flash-attn

🔹 High-Performance:
   llama-server -m {model_path} \\
       --host 0.0.0.0 --port 8080 \\
       --n-gpu-layers 99 --flash-attn \\
       --ctx-size 8192 --batch-size 1024 \\
       --parallel 4 --cont-batching \\
       --threads 4 --threads-batch 4

🔹 With Embeddings:
   llama-server -m {model_path} \\
       --host 0.0.0.0 --port 8080 \\
       --n-gpu-layers 99 --flash-attn \\
       --embeddings
"""

print(cli_examples)

📋 COMMAND-LINE REFERENCE

🔹 Basic Start:
   llama-server -m /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf --host 0.0.0.0 --port 8080

🔹 Single GPU (GPU 0, 15GB):
   llama-server -m /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf \
       --host 0.0.0.0 --port 8080 \
       --n-gpu-layers 99 --main-gpu 0 \
       --ctx-size 4096 --flash-attn

🔹 Dual GPU (30GB total):
   llama-server -m /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf \
       --host 0.0.0.0 --port 8080 \
       --n-gpu-layers 99 \
       --tensor-split 0.5,0.5 \
       --split-mode layer \
       --ctx-size 8192 --flash-attn

🔹 High-Performance:
   llama-server -m /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf \
       --host 0.0.0.0 --port 8080 \
       --n-gpu-layers 99 --flash-attn \
       --ctx-size 8192 --batch-size 1024 \
       --parallel 4 --cont-batching \
       --threads 4 --threads-batch 4

🔹 With Embeddings:
   llama-server -m /kaggle/working/models/gemma-3-1b-it-Q4_K_M.gguf \
       --host 0.0.0.0 -

## Step 13: Cleanup and Resource Release

Stops server, releases GPU memory, and verifies clean shutdown with final GPU status check for resource cleanup validation.

In [21]:
# Stop server
print("🛑 Stopping server...")
server.stop_server()

print("\n✅ Server stopped. Resources freed.")

# Final GPU status
print("\n📊 Final GPU Memory Status:")
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

🛑 Stopping server...

✅ Server stopped. Resources freed.

📊 Final GPU Memory Status:
index, memory.used [MiB], memory.free [MiB]
0, 0 MiB, 14913 MiB
1, 0 MiB, 14913 MiB


## 📚 Summary

You've learned:
1. ✅ Server configuration options
2. ✅ Kaggle T4 presets (single/dual GPU)
3. ✅ High-performance tuning
4. ✅ Health monitoring
5. ✅ Command-line reference

## Configuration Tips for Kaggle T4

| Model Size | Quantization | VRAM | Context | Config |
|------------|--------------|------|---------|--------|
| 1-3B | Q4_K_M | ~2GB | 8192 | Single T4 |
| 4-7B | Q4_K_M | ~5GB | 4096 | Single T4 |
| 8-13B | Q4_K_M | ~8GB | 4096 | Dual T4 |
| 13-30B | IQ3_XS | ~12GB | 2048 | Dual T4 |

---

**Next:** [03-multi-gpu-inference](03-multi-gpu-inference-llamatelemetry-v0.1.0.ipynb)